In [ ]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = 'mypassword'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

# Automated Trend Detection in Sales Data

## Description of the use case
This use case involves detecting trends in sales data using Redis T-Digest. By analyzing the distribution of sales data, businesses can identify patterns and make data-driven decisions.

## Redis Data Structures Used
- **T-Digest**: To detect trends in sales data.
- **Hashes**: To store sales data for each product.


In [ ]:
# Mock data for the use case
sales_data = {
    'product_1001': [5000, 5200, 5300, 5400, 5600],
    'product_1002': [7500, 7600, 7700, 7800, 7900],
    'product_1003': [3000, 3200, 3400, 3600, 3800],
    'product_1004': [2000, 2200, 2400, 2600, 2800]
}



# Use Hashes to store sales data for each product
for product_id, data in sales_data.items():
    r.hset(product_id, mapping={'sales_data': ','.join(map(str, data))})

# Use T-Digest to detect trends in sales data
for product_id, data in sales_data.items():
    r.tdigest().create(key=f'tdigest:{product_id}')
    for value in data:
        r.execute_command('TDIGEST.ADD', f'tdigest:{product_id}', value)

# Function to get percentiles from T-Digest
def get_percentiles(product_id, percentiles):
    return r.execute_command('TDIGEST.QUANTILE', f'tdigest:{product_id}', *percentiles)

# Detect trends in sales data for each product
percentiles_to_check = [0.25, 0.5, 0.75, 0.99]  # 25th, 50th, 75th and 99th percentiles
for product_id in sales_data.keys():
    percentiles = get_percentiles(product_id, percentiles_to_check)
    print(f'Trend for {product_id}:')
    for p, value in zip(percentiles_to_check, percentiles):
        print(f' > {int(p*100)}th percentile: {value}')

# Print the sales data for each product
for product_id in sales_data.keys():
    print(f"{product_id}: {r.hget(product_id, 'sales_data')}")
